# Experiment 3, Model 2

#### Model Setup

Run models in the following order, using their output labels as features for the next model:

1. Multiclass Person Name + Occupation Sequence Classifier

2. Multilabel Document Classifier

Train the first model and then run it over the entire dataset.

***

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/experiment_input/`
    * Prediction Data: Data: under directory `../data/token_clf_data/model_output/experiment3/`
* Word Embeddings
    * Custom fastText (word2vec with subwords) embeddings of 100 dimensions trained on the CRC Archives catalog's descriptive metadata (harvested October 2020)
    
***

**Table of Contents**

[I.](#i) Stereotype + Omission Classifier
* [Preprocessing](#prep)
* [Training & Prediction](#tp)
* [Evaluation](#eval)

[II.](#ii) Predict Over All Data

Load programming resources:

In [1]:
# For custom functions and variables
import utils, utils1, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For classification
import scipy
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

# For saving model
from joblib import dump,load

Define resources for the models:

In [2]:
Path(config.experiment_input_path).mkdir(parents=True, exist_ok=True)    # For train, devtest, and blind test data
# Path(config.experiment1_output_path).mkdir(parents=True, exist_ok=True)  # For predictions
# Path(config.experiment1_agmt_path).mkdir(parents=True, exist_ok=True)    # For agreement metrics

predictions_dir = config.experiment3_path+"5fold/output/"      # For predictions
Path(predictions_dir).mkdir(parents=True, exist_ok=True)
agreement_dir = config.experiment3_path+"5fold/agreement/"     # For agreement metrics
Path(agreement_dir).mkdir(parents=True, exist_ok=True)

# predictions_dir = config.experiment3_path+"5fold_withdescid/output/"      # For predictions
# Path(predictions_dir).mkdir(parents=True, exist_ok=True)
# agreement_dir = config.experiment3_path+"5fold_withdescid/agreement/"     # For agreement metrics
# Path(agreement_dir).mkdir(parents=True, exist_ok=True)

# predictions_dir = config.experiment3_path+"5fold/with_manual_labels/output/"              # For predictions with features as manual labels
# Path(predictions_dir).mkdir(parents=True, exist_ok=True)  
# agreement_dir = config.experiment3_path+"5fold/with_manual_labels/agreement/"             # For agreement metrics with features as manual labels
# Path(agreement_dir).mkdir(parents=True, exist_ok=True)

In [3]:
# Model 2:
pers_o_label_subset = ["B-Unknown", "I-Unknown", "B-Feminine", "I-Feminine", "B-Masculine", "I-Masculine", "B-Occupation", "I-Occupation"]
# Model 3:
so_label_subset = ["B-Stereotype", "I-Stereotype", "B-Omission", "I-Omission"]

In [4]:
pers_o_label_tags = {
    "Unknown": ["B-Unknown", "I-Unknown"], "Feminine": ["B-Feminine", "I-Feminine"], "Masculine": ["B-Masculine", "I-Masculine"],
     "Occupation": ["B-Occupation", "I-Occupation"]
    }
so_label_tags = {
    "Stereotype": ["B-Stereotype", "I-Stereotype"], "Omission": ["B-Omission", "I-Omission"]
             }

In [5]:
d = 100               # dimensions of word embeddings (should match utils1.py) for file names
target_labels = "so"  # for file names

<a id="i"></a>
## I. Stereotype + Omission Classifier
<a id="a"></a>
### Preprocessing

Load the document classification model's input data:

In [6]:
# For modified 5-fold cross validation
df = pd.read_csv(config.tokc_path+"experiment_input/document_5fold.csv", index_col=0)
df_exp = utils.getColumnValuesAsLists(df, "label")
df_exp = df_exp.drop(columns=["subset"])
df_exp.head()

,description_id,start_offset,end_offset,field,description,label,fold
0,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",[Omission],split3
1,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,[],split2
2,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,[],split0
3,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,"[Omission, Stereotype]",split0
4,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,[Omission],split3


Load the Person Name and Occupation labels as features and associate description IDs to the data, creating one row per description ID:

In [7]:
feat1_col = "tag_pers_o_predicted"  #"label_pers_o_predicted"  #"label_pers_o_withdescid_predicted" #"label_pers_o_expected"

In [8]:
## Use Person Name and Occupation BI tags as features, generalized to the label at the description level
features_filename = "crf_{a}_{t}_baseline_fastText{d}_predictions.csv".format(a="arow", d=d, t="pers_o")
df_features = pd.read_csv(config.experiment3_path+"5fold/output/"+features_filename, usecols=["sentence_id", "token_id", feat1_col])
df_features = df_features.rename(columns={"tag_pers_o_predicted":"pers_o_pred"})
# Generalize person name and occupation tags to labels
for label,tags in pers_o_label_tags.items():
    for tag in tags:
        df_features["pers_o_pred"] = df_features["pers_o_pred"].replace(to_replace=tag, value=label)
df_features.head()

,sentence_id,token_id,pers_o_pred
0,8,233,Masculine
1,8,234,Masculine
2,8,235,O
3,8,236,O
4,8,237,O


In [9]:
feat1_col = "pers_o_pred"

In [10]:
df_desc = pd.read_csv(config.agg_path+"descs_sents_tokens_anns.csv", usecols=["description_id", "sentence_id", "token_id"])
df_desc = df_desc.set_index("description_id")
df_desc = utils1.getColumnValuesAsLists(df_desc, "sentence_id")
df_desc = utils1.getColumnValuesAsLists(df_desc, "token_id")
df_desc_exploded = df_desc.explode(["sentence_id", "token_id"])
df_desc_exploded = df_desc_exploded.reset_index()
df_desc_exploded = df_desc_exploded.astype("int64")
# df_desc_exploded.head()

In [11]:
joined = df_features.join(df_desc_exploded.set_index(["sentence_id", "token_id"]), on=["sentence_id", "token_id"])
joined[feat1_col] = joined[feat1_col].fillna("O")
grouped = utils.implodeDataFrame(joined, ["description_id"]).reset_index()
# grouped.head()

Join the Person Name + Occupation feature column to the document classification model data:

In [12]:
features = grouped[["description_id", feat1_col]]
join_on = "description_id"
df = df_exp.join(features.set_index(join_on), on=join_on)
df = df.loc[~df.description.isna()]
df.head()

,description_id,start_offset,end_offset,field,description,label,fold,pers_o_pred
0,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",[Omission],split3,"[O, O, O, O, O, O, Unknown, O, O, O, Unknown, ..."
1,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,[],split2,"[O, O, O, O, Unknown, Unknown, O, O, O, O, O, ..."
2,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,[],split0,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,"[Omission, Stereotype]",split0,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,[Omission],split3,"[Masculine, O, O, O, O, O, O, O, O, O, O, O, U..."


Get unique values for feature lists with Person Name and Occupation labels:

In [13]:
perso_features = list(df[feat1_col])
# Get non-repeating lists of Person Name and Occupation labels
new_perso = [list(set(labels)) for labels in perso_features]
# Remove 'O' from features if other Person Name or Occupation labels are present
new_perso_features = []
for features in new_perso:
    if (len(features) > 1) and ("O" in features):
        features.remove("O")
    new_perso_features += [features]
df = df.drop(columns=[feat1_col])
df.insert(len(df.columns), feat1_col, new_perso_features)
df.head()

,description_id,start_offset,end_offset,field,description,label,fold,pers_o_pred
0,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",[Omission],split3,[Unknown]
1,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,[],split2,[Unknown]
2,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,[],split0,"[Occupation, Unknown]"
3,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,"[Omission, Stereotype]",split0,"[Feminine, Unknown, Occupation, Masculine]"
4,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,[Omission],split3,"[Masculine, Unknown]"


Define the train (80% of the data) and test (20% of the data) splits:

In [14]:
split_col = "fold"
splits = df[split_col].unique()
splits.sort()
print(splits)
train0, test0 = list(splits[:4]), splits[4]
train1, test1 = list(splits[1:]), splits[0]
train2, test2 = list(splits[2:])+[splits[0]], splits[1]
train3, test3 = list(splits[3:])+list(splits[:2]), splits[2]
train4, test4 = [splits[4]]+list(splits[:3]), splits[3]
runs = [(train0, test0), (train1, test1), (train2, test2), (train3, test3), (train4, test4)]
for run in runs:
    print(run)

['split0' 'split1' 'split2' 'split3' 'split4']
(['split0', 'split1', 'split2', 'split3'], 'split4')
(['split1', 'split2', 'split3', 'split4'], 'split0')
(['split2', 'split3', 'split4', 'split0'], 'split1')
(['split3', 'split4', 'split0', 'split1'], 'split2')
(['split4', 'split0', 'split1', 'split2'], 'split3')


In [15]:
def binarizeMultilabelTrainColumn(df_col):
    mlb = MultiLabelBinarizer()
    binarized = mlb.fit_transform(df_col)
    return mlb, binarized

def binarizeMultilabelDevColumn(mlb, df_col):
    binarized = mlb.transform(df_col)
    return binarized

<a id="tp"></a>
### Train & Predict

In [16]:
a = "sgd-svm"

In [17]:
pred_df = pd.DataFrame()
target_col = "label"
# feat1_col = "doc_"+feat1_col
for run in runs:
    # Get the train (80%) and test (20%) subsets of data
    train_splits, test_split = run[0], run[1]
    print("Training on:", train_splits)
    train_df = df.loc[df[split_col].isin(train_splits)]
    dev_df = df.loc[df[split_col] == test_split]
    
    # Binarize the features
    mlb_feat, train_feat = binarizeMultilabelTrainColumn(train_df[feat1_col])
    dev_feat = binarizeMultilabelDevColumn(mlb_feat, dev_df[feat1_col])
    
    # Vectorize the documents (descriptions)
    cvectorizer = CountVectorizer()
    tfidf = TfidfTransformer()
    train_docs = cvectorizer.fit_transform(train_df["description"])
    dev_docs = cvectorizer.transform(dev_df["description"])
    train_docs = tfidf.fit_transform(train_docs)
    dev_docs = tfidf.transform(dev_docs)
    
    # Concatenate the features and documents
    X_train = scipy.sparse.hstack([train_docs, train_feat])
    X_dev = scipy.sparse.hstack([dev_docs, dev_feat])
    
    # Binarize targets
    mlb_target, y_train = binarizeMultilabelTrainColumn(train_df["label"])
    y_dev = binarizeMultilabelDevColumn(mlb_target, dev_df["label"])

    # Train a classification model
    clf = OneVsRestClassifier(SGDClassifier(loss="hinge"))  # Support Vector Machines loss function
    clf.fit(X_train, y_train)
    
    # Predict with the trained model
    print("Predicting on:", test_split)
    predictions = clf.predict(X_dev)
    pred_labels = mlb_target.inverse_transform(predictions)    
    if pred_df.shape[0] > 0:
        next_pred_df = dev_df.copy()
        next_pred_df.insert(len(next_pred_df.columns), "predicted_label", pred_labels)
        pred_df = pd.concat([pred_df, next_pred_df])
    else:
        pred_df = dev_df.copy()
        pred_df.insert(len(pred_df.columns), "predicted_label", pred_labels)

print("Modified 5-fold cross-validation complete!")
print(pred_df.shape)

Training on: ['split0', 'split1', 'split2', 'split3']
Predicting on: split4
Training on: ['split1', 'split2', 'split3', 'split4']
Predicting on: split0
Training on: ['split2', 'split3', 'split4', 'split0']
Predicting on: split1
Training on: ['split3', 'split4', 'split0', 'split1']
Predicting on: split2
Training on: ['split4', 'split0', 'split1', 'split2']
Predicting on: split3
Modified 5-fold cross-validation complete!
(27312, 9)


Save the model:

In [18]:
# Model run with PN+O labels as features
model_dir = "models/experiment3/"
Path(model_dir).mkdir(parents=True, exist_ok=True)
filename = model_dir+"sgd-svm_F-fastText100-pers-o_T-s-o.joblib"
dump(clf, filename)

['models/experiment3/sgd-svm_F-fastText100-pers-o_T-s-o.joblib']

In [19]:
pred_df = pred_df.rename(columns={"label":"expected_label"})
pred_df.head()

,description_id,start_offset,end_offset,field,description,expected_label,fold,pers_o_pred,predicted_label
6,3027,627,1162,Biographical / Historical,Thomas Young was probably born in 1725. By the...,[Stereotype],split4,"[Masculine, Occupation, Unknown]","(Stereotype,)"
7,3397,8095,8334,Biographical / Historical,Andrew Tait worked on Paramecium in Beale's la...,[Omission],split4,"[Occupation, Masculine, Unknown]","(Omission,)"
10,4736,9951,10026,Biographical / Historical,Delivered by Thomson to teachers in Darlington.,[Omission],split4,"[Occupation, Unknown]","(Omission,)"
14,4712,4199,4485,Biographical / Historical,"This was gifted by Thomson to his secretary, M...",[Omission],split4,"[Occupation, Feminine, Unknown]","(Omission,)"
22,15684,845,1179,Biographical / Historical,Catherine Robina Borland was responsible for t...,[],split4,[Occupation],"(,)"


Save the predictions data:

In [20]:
pred_df.to_csv(predictions_dir+"aggregated_final_pno_predictions_docclf_{a}_{t}.csv".format(a=a, t=target_labels))

### Peformance

Calculate performance metrics for the Stochastic Gradient Descent classifier

In [21]:
classes = clf.classes_  #doc_clf.classes_
print(classes)
original_classes = mlb_target.classes_
print(original_classes)
label_dict = dict(zip(original_classes, classes))

[0 1 2]
['' 'Omission' 'Stereotype']


Create a [confusion matrix](https://scikit-learn.org/stable/modules/model_evaluation.html#multilabel-confusion-matrix) of the results, where, for class *i*:
* Count of true negatives (TN) is at position *i*,0,0
* Count of false negatives (FN) is at position *i*,1,0
* Count of true positives (FP) is at position *i*,1,1
* Count of false positives (PF) is at position *i*,0,1

In [22]:
y_dev = binarizeMultilabelDevColumn(mlb_target, pred_df["expected_label"])
predictions = mlb_target.transform(pred_df["predicted_label"])
assert len(y_dev) == len(predictions)

In [23]:
matrix = multilabel_confusion_matrix(y_dev, predictions, labels=classes)

In [24]:
scores = utils.getPerformanceMetrics(y_dev, predictions, matrix, classes, original_classes, label_dict)
scores = scores.tail(2) # Remove row for 'None'
# scores = scores.drop(columns="true_neg")  # Not accurate because considers 'None' a class
scores["labels"] = original_classes[1:]
scores

,labels,true_neg,false_neg,true_pos,false_pos,precision,recall,f_1
1,Omission,22856,1728,2304,424,0.844575,0.571429,0.681657
2,Stereotype,25622,374,1227,89,0.932371,0.766396,0.841275


Save the performance results:

In [25]:
scores.to_csv(agreement_dir+"docclf_{a}_pno_{t}_baseline_performance.csv".format(a=a, t=target_labels))

***

#### *For train-dev-test (i.e., 60-20-20) approach*

<a id="ii"></a>
## II. Predict Over All Data

### Preprocessing
Vectorize the documents, and binarize the features and targets:

In [34]:
target_col = "label"
feat1_col = "doc_ling_pred"

In [35]:
y_all = binarizeMultilabelDevColumn(mlb_target, df["label"])
print(y_all.shape)

(27312, 3)


In [36]:
all_feat1 = binarizeMultilabelDevColumn(mlb_feat1, df[feat1_col])
print(all_feat1.shape)

(27312, 3)


In [37]:
all_docs = cvectorizer.transform(df["description"])
all_docs = tfidf.transform(all_docs)
print(all_docs.shape)

(27312, 26960)


In [38]:
all_feats = scipy.sparse.csr_matrix(all_feat1)

Concatenate the documents and features, creating one scipy sparse matrix for the train data and another for the dev data:

In [39]:
X_all = scipy.sparse.hstack([all_docs, all_feats])
print(X_all.shape)

(27312, 26963)


### Predict

In [40]:
predicted_all = doc_clf.predict(X_all)

### Peformance

Calculate performance metrics for the Stochastic Gradient Descent classifier

In [41]:
print("Accuracy:", np.mean(predicted_all == y_all))

Accuracy: 0.9497046475297793


In [42]:
matrix = multilabel_confusion_matrix(y_all, predicted_all, labels=classes)

In [43]:
scores = utils.getPerformanceMetrics(y_all, predicted_all, matrix, classes, original_classes, label_dict)
scores = scores.tail(2) # Remove row for 'None'
scores = scores.drop(columns="true_neg")  # Not accurate because considers 'None' a class
scores["labels"] = original_classes[1:]
scores

,labels,false_neg,true_pos,false_pos,precision,recall,f_1
1,Omission,1637,2395,252,0.904798,0.593998,0.717173
2,Stereotype,343,1258,48,0.963247,0.785759,0.865497


Save the performance results:

In [44]:
# dir_path = config.tokc_path+"/experiment2/output/"
scores.to_csv(dir_path+"docclf_{a}_{t}_baseline_performance_ALLDATA.csv".format(a=a, t=target_labels))

Add the predicted labels to the dev data:

In [45]:
pred_all_labels = mlb_target.inverse_transform(predicted_all)

Add the classifier's labels to the `aggregated_validate.csv` DataFrame of descriptions to facilitate error analysis:

In [46]:
df = df.rename(columns={"label":"manual_label"})
df.insert(len(df.columns), "{a}_label".format(a=a), pred_all_labels)
df.head()
# print(len(pred_all_labels), df.shape)

,description_id,start_offset,end_offset,field,description,subset,manual_label,doc_ling_pred,sgd-svm_label
4699,4699,1853,2066,Biographical / Historical,"Labelled Apparently some chapters, amounting t...",train,[Omission],[Gendered-Role],"(None,)"
8942,8942,384,540,Biographical / Historical,James Aikman of Perth signed his name to a vol...,train,[None],[Gendered-Pronoun],"(None,)"
5440,5440,5692,5850,Biographical / Historical,This piece was published in 'Milk Production i...,train,[None],[],"(None,)"
3474,3474,3608,8549,Biographical / Historical,Margaret Winifred Bartholomew was born on 21 A...,train,"[Omission, Stereotype]","[Generalization, Gendered-Role]","(Omission, Stereotype)"
4769,4769,2378,2576,Biographical / Historical,Blacker and Thomson became close friends throu...,train,[Omission],[Gendered-Pronoun],"(Omission,)"


Save this version of the data:

In [47]:
df.to_csv(dir_path+"aggregated_final_validate_predictions_docclf_{a}_{t}_ALLDATA.csv".format(a=a, t=target_labels))